In [ ]:
import numpy as np
import cudf
import cuspatial
from shapely.geometry import Point, Polygon
import geopandas
import time 



N = 10
lenpoly = 1000
N_polys = 40
polygon_list = [None]*N_polys
for j in range(N_polys):
    shift = np.random.uniform(-1,1)
    polygon = [[np.sin(x)+shift,np.cos(x)+shift] for x in np.linspace(0,2*np.pi,lenpoly)]
    polygon_arr = np.array(polygon)
    polygon_list[j] = polygon_arr

points = np.random.uniform(-1.5, 1.5, size=(N, 2))

st = time.time()
points_list = [Point(points[i,:]) for i in range(points.shape[0])]
shapekly_result_list = [None]*N_polys
for j in range(N_polys):
    shapely_poly = Polygon(polygon_list[j])
    shapely_result = shapely_poly.contains(points_list)
    shapekly_result_list[j] = shapely_result
end = time.time()

print(end-st)


st = time.time()

shapely_points_list = [Point(points[i,:]) for i in range(points.shape[0])]
shapely_poly_list = [Polygon(polygon) for polygon in polygon_list]

cupoints = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_points_list))
cupoly = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_poly_list))
current_index = 0
next_index = 0
while next_index < N_polys:
    if N_polys - current_index > 30:
        next_index = current_index + 30
    else:
        next_index = N_polys
    result = cuspatial.point_in_polygon(cupoints,
                                    cupoly[current_index:next_index]                   
                                    )
    current_index = next_index
end = time.time()

print(end-st)

st = time.time()
interleaved_points = points.flatten()
cupoints2 = cuspatial.GeoSeries.from_points_xy(interleaved_points)

shapely_poly = [Polygon(polygon) for polygon in polygon_list]
cupoly = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_poly))
print("length:",len(cupoly))


current_index = 0
next_index = 0
frames = []
while next_index < N_polys:
    if N_polys - current_index > 30:
        next_index = current_index + 30
    else:
        next_index = N_polys
    result = cuspatial.point_in_polygon(cupoints2,
                                    cupoly[current_index:next_index]                   
                                    )
    #df.update(result)
    current_index = next_index
    frames.append(result)
    
end = time.time()
print(end-st)

print(result)
result.loc[0]
result.values

df = cudf.concat(frames,axis=1)
print(df)
print(df.values[2,39])
type(df.values)

In [ ]:
import numpy as np
import cuspatial
from shapely.geometry import Point, Polygon

N=10

points = np.random.uniform(-1.5, 1.5, size=(N, 2))
shapely_points_list = [Point(points[i,:]) for i in range(points.shape[0])]
cupoints = cuspatial.GeoSeries(geopandas.GeoSeries(shapely_points_list))
print(cupoints)
print(cupoints[0:10])

cupoints.points


In [ ]:
import numpy as np

a = np.array([[[1,1,1],[2,2,2],[3,3,3],[4,4,4]],[[1,1,1],[2,2,2],[3,3,3],[4,4,4]],[[5,1,1],[2,2,2],[3,3,3],[4,6,4]]])
a.shape[1]

a[0:2]

b = np.reshape(a,(-1,3))
print(b)
print(b[:,2])

In [ ]:
from bisect import bisect_left


def take_closest(myList_org, myNumber):
    """
    Assumes myList is sorted. Returns index of closest value and the closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    myList = myList_org.copy()
    if myList[0] > myList[1]:
        myList.reverse()
        list_reversed = True
    else:
        list_reversed = False
    if list_reversed == False:
        pos = bisect_left(myList, myNumber)
        if pos == 0:
            return 0, myList[0]
        if pos == len(myList):
            return pos-1, myList[-1]
        before = myList[pos - 1]
        after = myList[pos]
        if after - myNumber < myNumber - before:
            return pos, after
        else:
            return pos - 1, before
    elif list_reversed == True:
        pos = bisect_left(myList, myNumber)
        if pos == 0:
            return len(myList)-1, myList[0]
        if pos == len(myList):
            return 0, myList[-1]
        before = myList[pos - 1]
        after = myList[pos]
        if after - myNumber < myNumber - before:
            return len(myList) - 1 - pos, after
        else:
            return len(myList) - pos, before


def take_closest_array_input(myList, myArray):
    """
    Assumes myList is sorted. Returns index of closest value and the closest value to myNumber.

    If two numbers are equally close, return the smallest number. Takes an array as input and vectorizes the 'take_closest_index_only' function. 

    Returns an array of the indices of the original list that contains the value that is closest to each element of the given array.
    """

    take_closest_index_only_vectorized = np.vectorize(take_closest, excluded=['myList_org'])
    closest_vals_indices_array, closest_vals_array = take_closest_index_only_vectorized(myList_org = myList,myNumber = myArray)
    return closest_vals_indices_array, closest_vals_array


a = [1,3,5,7,8,9,10,12,14,15,16,17,18,19]

b = np.array([3.4,5.6,4.8,8.8,1.5,6.3,6.4,10.7,19,13])

c, d = take_closest_array_input(a,b)

print(c)
print(d)



In [ ]:
import cupy as cp

a = [cp.array(True) for j in range(10)]

b = cp.array(a)

df = cudf.DataFrame.from_dict({"a":a})
b

color = cp.array([1,0,0],dtype=float)

color

In [ ]:
a = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

for j,i in enumerate(a):
    print(j % 10)


In [ ]:
import cudf
import numpy as np

N=10

points = [np.array([1,1,1]),np.array([1,2,1]),np.array([1,1,2])]

dictionary = {"points": points}

a = cudf.DataFrame.from_dict(dictionary)
a

In [ ]:
[int(i) for i in range(5) for j in range(3)]

In [ ]:
import numpy as np

a = np.array([[1,2,3],[4,5,6],[7,8,9]])
b=np.array([0,1])
c=np.array([1,2])
c = a[b,c]
c

In [ ]:
import numpy as np

def color_by_bool_by_arr(contain_bool_arr):
    color_by_bool_vectorized = np.vectorize(color_by_bool)
    color_arr = color_by_bool_vectorized(contain_bool_arr)
    return color_arr



def color_by_bool(bool_val):
    if bool_val == True:
        color = np.array([0,1,0],dtype=float)
    elif bool_val == False:
        color = np.array([1,0,0],dtype=float)

    return color

a = np.array([True, False, True])

for element in a:
    print(element)

In [ ]:
import cupy as cp 
import time  
import numpy as np


def color_by_bool_by_arr(contain_bool_arr):
    color_list = [None]*len(contain_bool_arr)
    for index,element_bool in enumerate(contain_bool_arr):
        if element_bool == True:
            color = np.array([0,1,0])
        else:
            color = np.array([1,0,0])
        color_list[index] = color
    return color_list

a = cp.random.randint(2, size=10000)
st = time.time()
b= color_by_bool_by_arr(a)

end = time.time()
print(end-st)
a = np.random.randint(2, size=10000)


st = time.time()
b= color_by_bool_by_arr(a)

end = time.time()
print(end-st)


In [ ]:
import cudf
import pandas
import numpy as np

example_dict = {"1":[True, False, True],"2":[False, False, True]}

df = cudf.DataFrame(example_dict)

print(df)

df.to_numpy()[[0,1,2],[1,1,1]]







In [ ]:
import numpy as np

a = np.empty([10,3])
a[2,:] = np.array([1,1,1])
a

In [ ]:
import cupy as cp 
import numpy as np

points = np.array([[[1,1,1],[1,1,1],[1,1,1]],[[2,2,2],[2,2,2],[2,2,2]]])

points_cp = cp.array(points)

vec_arr = np.array([[0.5,0.5,0.5],[1,1,1]])

vec_arr_cp = cp.array(vec_arr)

vec_arr_cp_reshaped = cp.reshape(vec_arr_cp,(2,1,3))

vec_arr_cp_reshaped

vec_arr_cp_extended = cp.tile(vec_arr_cp_reshaped,(1,3,1))

vec_arr_cp_extended

points_cp_translated = points_cp + vec_arr_cp_extended

points_cp_translated

In [ ]:
test = [[1,2,3],2,3]
test1 = test.copy()

def test_func(list):
    list.append(3)
    return 3

test_func(test1)

test1[0].append(1)
test


In [ ]:
import numpy as np

import cupy as cp

a = np.array([1,2,3])

b = np.array([1,2,3])

c = cp.array([1,2,3])

d = cp.array(a)

e = cp.array(c)


d[0] =10 

a




In [ ]:
import cupy as cp

a = cp.array([[1,1,1],[2,2,2],[3,3,3]])

b = cp.resize(a,(1,3,3))
c = cp.repeat(b,3,0)

d = cp.array([1,2,3,4])
print(d)
print(d.shape)
e=d[...,None]
print(e)
e.shape

In [ ]:
import numpy as np

a = np.array([1,2,3,4])
print(a.shape)
b=a[...,None]
print(b.shape)
b


In [ ]:
import cupy as cp
import numpy as np

a = cp.array([[[1,1,1],[1,1,1],[1,1,1]],[[2,2,2],[2,2,2],[2,2,2]]])

for slice in a:
    print(slice)
    print('___')

In [ ]:
print("org "+str(2-1))

In [ ]:
import numpy as np

a = np.array(None)
a

In [ ]:
import cudf 

df = cudf.DataFrame({'point idx': [0, 1, 2, 3, 0, 1, 2, 3],
                   'pt contained': [False, True, True, True, False, True, False, True]})
print(df)



a = df.groupby(['point idx']).sum().to_numpy().T
a


In [ ]:
import numpy as np

a = np.array([[1,1,1],[2,2,2]])
b = a.flatten(order = 'C').tolist()
b


In [ ]:
import cupy as cp   

a = cp.array([[[1,1,1],[1,1,1]],[[2,2,2],[2,2,2]]])
b= cp.reshape(a,(-1,3))
b

In [ ]:
import cupy as cp

a = cp.array([[1,4,3],[1,4,7],[3,10,15]])
b = (a[:,2] < 10) 
c = (a[:,2] > 5)
print(b)
print(c)
d = cp.logical_and(b,c)
d
e = [1,2,3,4,45,56]
max(e)


In [ ]:
import numpy as np

a = np.array([[1,2,3],[4,5,6],[7,8,9]])
a[0:2,0:2] = 10
a

In [ ]:
import pandas as pd


df = pd.DataFrame({
     'col_to_group': ['A', 'A', 'B', 'B', 'A', 'B'],
     'col_1': ['a', 'b', 'c', 'a', 'a', 'a'],
     'col_2': ['x', 'y', 'y', 'y','x','x'],
     'col_3': [.1, .2, .1, .9, .7,0.5],
     'col_4': ["Nominal", 1,2, "Nominal", 1, 2],
})

df3 = df[df['col_2']!= 'x'].reset_index()

print(df3)

df2 = df.drop([i for i in range(2)])

grouped = df.groupby(['col_4'])
df1 = grouped.get_group("Nominal")
df1

In [ ]:
import numpy as np

a = np.array([[[1,1,1],[1,1,1],[1,1,1]],[[2,2,2],[2,2,2],[2,2,2]]])
print(a)
print(a.shape[0])
b = np.array([[0,0,0],[0,0,0],[0,0,0]])
b = np.expand_dims(b,axis=0)
print(b)
c = np.concatenate((b,a))
print(c)



In [ ]:
import numpy as np

a = np.array([[1,1,1],[2,2,2],[3,3,3]])
b = np.array([4,4,4])
c = np.expand_dims(b,axis =1)
d = np.append(c,a, axis = 1)
print(d)
d.tolist()


In [ ]:
list(range(0,5))

In [ ]:
import numpy as np

a=np.arange(0,10).tolist()
a.prepend('1')

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
                go.Scatter(
                    name='Nominal',
                    x=[0,1,2,3,4],
                    y=[1,1,1,2,2],
                    marker=dict(color='red',
                                ),
                    #marker_line_width = 1,
                    
                    mode='markers',
                    showlegend=True
                )
            )

fig.show()

In [ ]:
import numpy as np


a = np.array([1,2,3]).tolist()
a

In [ ]:
import numpy as np

a = np.array([[1,2,3],[4,5,6],[7,8,9]])
b = np.array([11,12,13])
c = np.expand_dims(b,axis =1 )
d = np.append(a,c,axis = 1)
print(d)

e = np.empty((a.shape[0],a.shape[1]+1))
e[:,0:3] = a
e[:,3] = b
print(e)

e[:,3]


In [ ]:
list(range(0,3))

In [ ]:
import numpy as np

a = np.empty((3,2,4))
a.fill(1)
print(a)
a[0,:,:]

In [ ]:
a = {}

a["1"] = 1
a
len(a)

In [ ]:
import numpy as np

a = np.array([1,1,1])
b=np.array([2,2,2])
c=np.array([1,2,3,4])
d=np.array([[1,2,3],[1,2,3]])
print(a.shape == a.shape)
print(a.shape == b.shape)
print(a.shape == c.shape)
print(a.shape == d.shape)
len(d)
print((a - 2))
print((a - 2)**2)
print((b - 2)**2)
np.sum((a - 2)**2 + (b - 2)**2)


In [ ]:
a = {}
a['a'].append(1)

In [ ]:
a = np.empty(10)
a.shape[0]

In [ ]:
import numpy as np

a = np.array([1,2,3])
(a-1)**2

In [ ]:
1/2*3

In [ ]:
a = [1,2,3]
b = [4,5,6]
c = np.empty((2,1,3))
c[0] = a
c[1] = b
c
d = np.array(a).reshape((1,-1))
d[:,0:2]

In [ ]:
test_dict_1 = {'a':[1,2,3], 'b':[4,5,6]}

for index,(key,item) in enumerate(test_dict_1.items()):
    print(index)
    print(key)
    print(item)
    print('---')

len(test_dict_1)

In [ ]:
import pandas as pd
import numpy as np
import cudf

technologies = {
    'ROI':["P","P","D1","D1","D2","D2","P","P","D1","D1","D2","D2","P","P","D1","D1","D2","D2",],
    'Trial':[1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3],
    'original_pt' :[1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2],
    'Contained':[True,True,True,True,True,True,True,True,False,False,False,False,False,True,True,True,False,True],
    'test': np.array([True,True,True,True,True,True,True,True,False,False,False,False,False,True,True,True,False,True])
    }
df = pd.DataFrame(technologies)
print(df)
comb = ['P',"D1"]
"""
df[(df["ROI"].isin(comb))].reset_index()[
        ["Contained","original_pt","Trial"]
        ].groupby(["original_pt","Trial"]).all().sort_index().groupby(["original_pt"]).sum().sort_index().to_numpy().T.flatten(order = 'C').tolist()
"""
df['ROI'].unique().tolist()

In [ ]:
from itertools import combinations

a = [1,2,3]

res = [com for sub in range(1,3) for com in combinations(a , sub + 1)]
for comb in res:
    print(comb)
    print(1 in comb)
    print('---')



In [ ]:
import numpy as np

a = np.empty((10))
a.fill(0)
a

In [ ]:
import numpy as np

a = np.array([[1,1,1],[1,0,2]])
np.linalg.norm(a, axis=0, keepdims=False)

In [ ]:
test = 1.15
se = np.array([1,1,1])
vals=np.array([1,2,3])
low = vals - se*test
high = vals + se*test
print(low)
print(high)
result = np.array([low,high])
print(result)


In [ ]:
a = np.array([[1,0,3],[1,0,0]])
 
np.any(a,axis = 0)

In [ ]:
'prostate' in 'prostate 1'
'prostate 1' in 'prostate'

In [ ]:
a = np.array([[1,2,3,4],[1,2,3,4]])
1-a

In [ ]:
a= [(1,2),(3,4),(5,6)]
np.array(a).T

In [ ]:
import numpy as np 

a = np.array([[1,1,1],[2,2,2]])

a[[0,1]] = a[[1,0]]

a

In [ ]:
import numpy as np 

a = np.array([[1,2,3],[3,1,2],[3,4,9],[30,10,20]])

a[np.array([0,3]),2]    

#np.mean(a,axis=0)

In [ ]:
import pandas as pd

a = pd.DataFrame({'A':np.array([1,2,3]),'b':'Prostate'})
b = pd.DataFrame({'A':np.array([1,2,3]),'b':'DIL'})

frames = [a, b]

result = pd.concat(frames, ignore_index=True)
result

In [ ]:
import numpy as np 


a = np.array([[1,2,3],[4,5,6]])
a.flatten(order = 'C')
a[:,0]

In [ ]:
a = {1: False, 2: False, 3: False}
any(a.values())

In [ ]:
import cudf
df = cudf.DataFrame({'a': [1, 2, None], 'b': [3, None, 5]})
df
df_new = df.fillna(0)
print(df)
print(df_new)

d = df_new['a'].unique()
e = d.to_arrow().to_pylist()
e

In [ ]:
a = {'a': [1,2,3], 'b': 2, 'c': 'string'}
list(a.values())

In [ ]:
import pandas as pd
import numpy as np

a = {'a': [1,2,3], 'b': [4,5,6]}
df = pd.DataFrame(data = a).T
df

a = np.empty((1))
b= np.empty((3))
c = np.append(a,b)
a

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

df = px.data.tips()
print(df)

fig = px.box(df, x="day", y="total_bill", color="smoker")
fig2 = px.box(df, x="day", y="tip", color="smoker")
"""
for i in range(0,len(fig2.data)):
    fig.add_trace(fig2.data[i])
fig.update_layout(
        boxmode='group'
    )
"""

fig.update_traces(orientation='h') # horizontal box plots

fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()
"""
fig = go.Figure()
fig.add_trace(go.Box(
        x=df['day'],
        y=df["total_bill"]    ))
fig.show()
"""

In [ ]:
import cupy as cp 
import numpy as np

a = np.array([1,2,3])
b = cp.asnumpy(a)

In [ ]:
a = [1,2,3]
import numpy as np

np.std(a)
a.append('b')
a
'b' in a

In [ ]:
c = {}
c["2"].append(1)
c

In [1]:
a = list(['string'])
a.append('hello')
a

['string', 'hello']

In [5]:
a = [False,True]
int(False)

0

In [17]:
import numpy as np
a = np.array([[9, 2, 3],[4, 5, 6],[7, 0, 5]])
print(a)
print(a[0,:])
print(a[0,:].argsort(axis=0))
a = a[:,a[0,:].argsort(axis=0)] 
print(a)


[[9 2 3]
 [4 5 6]
 [7 0 5]]
[9 2 3]
[1 2 0]
[[2 3 9]
 [5 6 4]
 [0 5 7]]


In [29]:
import numpy as np

lattice_spacing = 0.5
threshold = 0.9
n = 10
z_coords = np.array([0.1,0.2,0.3,4,5,6,7,8,9,10,11,12])
binom_ests = np.array([0.95,0.95,0.8,0.9,0.9,0.9,0.9,0.4,0.3,0.2,0.9,0.9])
binom_se = np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])

data = np.array([z_coords,binom_ests,binom_se])
print(data)
sorted_data = data[:,data[0,:].argsort(axis=0)] 
print(sorted_data)

sample_matrix = np.empty([n,sorted_data.shape[1]])

for i in range(sample_matrix.shape[0]):
    for j in range(sample_matrix.shape[1]):
        sample_matrix[i,j] = np.random.normal(loc = sorted_data[1,j], scale = sorted_data[2,j])

print(sample_matrix)
length_estimate_dist = np.empty(sample_matrix.shape[0])
for i in range(sample_matrix.shape[0]):
    particular_sample_1darr = sample_matrix[i]
    delta_z_tot = 0
    z_start_found = False
    for index,binom_est in enumerate(particular_sample_1darr):
        if binom_est >= threshold:
            if z_start_found == True:
                pass
            else: 
                z_start = sorted_data[0][index]
                z_start_found = True
        else: 
            if z_start_found == True:
                z_end = sorted_data[0][index-1]
                z_diff = z_end - z_start
                if z_diff > lattice_spacing:
                    delta_z_tot = delta_z_tot + z_diff
                else:
                    delta_z_tot = delta_z_tot + lattice_spacing
                z_start_found = False
            else:
                pass
    
    length_estimate_dist[i] = delta_z_tot

print(length_estimate_dist)

[[ 0.1   0.2   0.3   4.    5.    6.    7.    8.    9.   10.   11.   12.  ]
 [ 0.95  0.95  0.8   0.9   0.9   0.9   0.9   0.4   0.3   0.2   0.9   0.9 ]
 [ 0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1 ]]
[[ 0.1   0.2   0.3   4.    5.    6.    7.    8.    9.   10.   11.   12.  ]
 [ 0.95  0.95  0.8   0.9   0.9   0.9   0.9   0.4   0.3   0.2   0.9   0.9 ]
 [ 0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1   0.1 ]]
[[0.95798322 1.02028378 0.81430878 0.9132312  0.95801417 0.89870017
  1.01172128 0.45926996 0.21703184 0.21422721 0.80625147 0.84884647]
 [0.80842082 1.07192684 0.59337478 1.05355372 1.00333941 0.99025672
  0.8436267  0.26623055 0.26176176 0.20293641 0.8305498  0.89087674]
 [0.95059275 0.85086578 0.85718504 1.06083162 1.03108033 1.08655145
  0.80710443 0.44316525 0.24001044 0.22152431 0.89695733 1.00777078]
 [0.99619048 1.04845777 0.82975674 0.8115122  0.82194788 0.78252474
  0.96531598 0.22323226 0.21186999 0.24050662 0.89924261 0.97946818]


In [25]:
a = np.array([4,5,6])
for index,obj in enumerate(a):
    print(index)
    print(obj)

0
4
1
5
2
6


In [2]:
a = {0.95: [1,2,3]}
a

{0.95: [1, 2, 3]}

In [3]:
import numpy as np
a = np.array([1,2,3,4,5])
b = np.array([1,2,3,4,5])
a-b
np.mean(a-b)

0.0

In [10]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1)

fig.append_trace(go.Scatter(
    x=[3, 4, 5],
    y=[1000, 1100, 1200],
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=[2, 3, 4],
    y=[100, 110, 120],
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=[0, 1, 2],
    y=[10, 11, 12]
), row=3, col=1)


fig.update_layout(height=600, width=600, title_text="Stacked Subplots")
fig.show()